# IESB - CIA035 - Aula 13 - NN Customer Churn

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv


In [2]:
# Carregando os dados
df = pd.read_csv('/kaggle/input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv')

df.shape

(7043, 21)

In [3]:
# Olhando os dados
df.head().T

,0,1,2,3,4
customerID,7590-VHVEG,5575-GNVDE,3668-QPYBK,7795-CFOCW,9237-HQITU
gender,Female,Male,Male,Male,Female
SeniorCitizen,0,0,0,0,0
Partner,Yes,No,No,No,No
Dependents,No,No,No,No,No
tenure,1,34,2,45,2
PhoneService,No,Yes,Yes,No,Yes
MultipleLines,No phone service,No,No,No phone service,No
InternetService,DSL,DSL,DSL,DSL,Fiber optic
OnlineSecurity,No,Yes,Yes,Yes,No


In [4]:
# % de clientes que deixaram a operadora
df['Churn'].value_counts(normalize=True)

No     0.73463
Yes    0.26537
Name: Churn, dtype: float64

In [5]:
# Verificando os tipos dos dados e os tamanhos
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [6]:
# Para corrigir o TotalCharges vamos trocar o espaço em branco
# pelo valor ZERO e forçar a conversão para float
df['TotalCharges'] = df['TotalCharges'].str.replace(' ', '0').astype(float)

In [7]:
# Convertendo as colunas categórias em colunas numéricas
for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = df[col].astype('category').cat.codes

In [8]:
# Verificando os tipos dos dados e os tamanhos
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   int16  
 1   gender            7043 non-null   int8   
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   int8   
 4   Dependents        7043 non-null   int8   
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   int8   
 7   MultipleLines     7043 non-null   int8   
 8   InternetService   7043 non-null   int8   
 9   OnlineSecurity    7043 non-null   int8   
 10  OnlineBackup      7043 non-null   int8   
 11  DeviceProtection  7043 non-null   int8   
 12  TechSupport       7043 non-null   int8   
 13  StreamingTV       7043 non-null   int8   
 14  StreamingMovies   7043 non-null   int8   
 15  Contract          7043 non-null   int8   
 16  PaperlessBilling  7043 non-null   int8   


In [9]:
# Importando o PyTorch
import torch
import torch.nn as nn

In [10]:
# Importando o train_test_split
from sklearn.model_selection import train_test_split

# Preparar e separar os dataframes
X = df.drop(['customerID', 'Churn'], axis = 1).values
y = df['Churn'].values

# Separando os dados
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Transformando os dados em tensores
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

In [11]:
# Visualizando os dados de treino
X_train[0:5]

tensor([[0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00, 2.1000e+01, 1.0000e+00,
         0.0000e+00, 0.0000e+00, 2.0000e+00, 0.0000e+00, 2.0000e+00, 0.0000e+00,
         0.0000e+00, 2.0000e+00, 1.0000e+00, 0.0000e+00, 3.0000e+00, 6.4850e+01,
         1.3368e+03],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 5.4000e+01, 1.0000e+00,
         2.0000e+00, 1.0000e+00, 0.0000e+00, 2.0000e+00, 0.0000e+00, 0.0000e+00,
         2.0000e+00, 2.0000e+00, 2.0000e+00, 1.0000e+00, 0.0000e+00, 9.7200e+01,
         5.1295e+03],
        [1.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00,
         1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00, 2.0000e+00, 2.3450e+01,
         2.3450e+01],
        [1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 4.0000e+00, 1.0000e+00,
         0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00

In [12]:
# Visualindos as resposta de treino
y_train[0:5]

tensor([0, 0, 1, 1, 0])

In [13]:
# Definindo uma rede neural (fully connected)
# Input layer: 19 entradas (variáveis de entrada) -> 16 saídas (arbitrário/definido por nós)
# Hideen layer: 16 entradas (combinando com as saídas da camada anterior) -> 12 saídas (arbitrário/definido por nós)
# Hideen layer: 12 entradas (combinando com as saídas da camada anterior) -> 6 saídas (arbitrário/definido por nós)
# Output layer: 6 entradas (combinando com as saídas da camada anterior) - 2 saídas (Churn 1 ou Churn 0)

# Criando a rede neural usando nn.Sequential
model = nn.Sequential(nn.Linear(19, 16),
                      nn.ReLU(),
                      nn.Linear(16, 12),
                      nn.ReLU(),
                      nn.Linear(12, 6),
                      nn.ReLU(),
                      nn.Linear(6, 2))

model

Sequential(
  (0): Linear(in_features=19, out_features=16, bias=True)
  (1): ReLU()
  (2): Linear(in_features=16, out_features=12, bias=True)
  (3): ReLU()
  (4): Linear(in_features=12, out_features=6, bias=True)
  (5): ReLU()
  (6): Linear(in_features=6, out_features=2, bias=True)
)

In [14]:
# Temos que definir a função de erro e o otimizador (qu vai alterar os pesos dos perceptrons)
error_function = nn.CrossEntropyLoss() # criterion
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [15]:
# Treinamento do modelo

# Definindo o número de épocas
epochs = 5000

# Erros
running_loss = []

# For para rodar o número de épocas
for i in range(epochs):
    # Treinamento
    
    # Foward Propagation (passando os dados de treino pela rede)
    outputs = model.forward(X_train)
    # Calculando o erro
    loss = error_function(outputs, y_train)
    # Guadando o erro
    running_loss.append(loss)
    
    # Exibindo o erro de 5 em 5 épocas
    if i % 10 == 0:
        print(f'Epoch: {i} / Loss: {loss}')
        
    # Back Propagation
    # Limpar os parametros do otimizador (zerar o Gradiente Descendent)
    optimizer.zero_grad()
    # Calcular os novos pesos
    loss.backward()
    # Executar o optimizador (efetivamente fazer o back propagation mudando os pesos)
    optimizer.step()

Epoch: 0 / Loss: 0.5977111458778381
Epoch: 10 / Loss: 0.5705974102020264
Epoch: 20 / Loss: 0.6440858840942383
Epoch: 30 / Loss: 0.598939061164856
Epoch: 40 / Loss: 0.5789453387260437
Epoch: 50 / Loss: 0.5805388689041138
Epoch: 60 / Loss: 0.578938901424408
Epoch: 70 / Loss: 0.5782160758972168
Epoch: 80 / Loss: 0.5782814621925354
Epoch: 90 / Loss: 0.5781020522117615
Epoch: 100 / Loss: 0.5780800580978394
Epoch: 110 / Loss: 0.5780349969863892
Epoch: 120 / Loss: 0.5780081748962402
Epoch: 130 / Loss: 0.5779757499694824
Epoch: 140 / Loss: 0.577954113483429
Epoch: 150 / Loss: 0.5779343843460083
Epoch: 160 / Loss: 0.5779356360435486
Epoch: 170 / Loss: 0.5779098868370056
Epoch: 180 / Loss: 0.5779094099998474
Epoch: 190 / Loss: 0.5779001712799072
Epoch: 200 / Loss: 0.5779052972793579
Epoch: 210 / Loss: 0.5778804421424866
Epoch: 220 / Loss: 0.5779090523719788
Epoch: 230 / Loss: 0.577887237071991
Epoch: 240 / Loss: 0.5779180526733398
Epoch: 250 / Loss: 0.5778601169586182
Epoch: 260 / Loss: 0.577871

In [16]:
# Verificando os 5 últimos valores da função de erro
running_loss[-5:]

[tensor(0.5779, grad_fn=<NllLossBackward>),
 tensor(0.5779, grad_fn=<NllLossBackward>),
 tensor(0.5779, grad_fn=<NllLossBackward>),
 tensor(0.5779, grad_fn=<NllLossBackward>),
 tensor(0.5779, grad_fn=<NllLossBackward>)]

In [17]:
# Previsões para os dados de teste

# Lista das previsões
preds = []

# Colocar a rede em modo de execução/previsão / tirar do modo de treinamento
with torch.no_grad():
    for val in X_test:
        predict = model.forward(val)
        preds.append(predict.argmax().item())

In [18]:
# Criando um dataframe com os resultados
df_result = pd.DataFrame({'Y': y_test, 'YHat': preds})
df_result['Correct'] = [1 if corr == pred else 0 for corr, pred in zip(df_result['Y'], df_result['YHat'])]

df_result.head()

,Y,YHat,Correct
0,1,0,0
1,0,0,1
2,0,0,1
3,1,0,0
4,0,0,1


In [19]:
# Medindo a acurácia
df_result['Correct'].sum() / len(df_result)

0.7352732434350603